In [135]:
from scripts.utils.utils import load_json
import pandas as pd
import numpy as np
from scripts.pruning_models.classification import apply_models
from scripts.pruning_models.model import calculate_reward_transition_matrices_new,calculate_q_matrix_avpruning,calculate_traces
from scripts.utils.utils import load_yaml
n_steps=8

In [114]:
networks = load_json('../data/test/networks/networks.json')
df = pd.read_parquet('../data/test/preselection/preselection.parquet.gzip')
#df_old=pd.read_parquet('/home/gezerli/Desktop/projects/reward_networks/data/exp1/preselection/preselection.parquet.gzip')
df_deneme = pd.read_parquet('../data/deneme/preselection/preselection.parquet.gzip')
params=load_yaml('../params/test/deneme.yml')

In [124]:
from itertools import product
def get_actions(n_steps):
    all_acts=[]
    for j in product((0,1), repeat=n_steps):
        all_acts.append(j)
    return all_acts

In [136]:
def make_row(network,solution_idx,RT,AT,NT,ORT,max_tot_reward):
    """
        Make dataframe rows for each environment for storage.

    """
    #environmentId, solutionIdx, AT, NT, RT, ORT, RT_tot, totalRegret.
    #new_id = hashlib.md5((network['network_id'] + str(n)).encode()).hexdigest()
    #solution_idx = str(uuid.uuid4())
    #model_parameter_capitalize = capitalize_keys(model_parameter)

    return {
        'Environment_ID': network['network_id'], 'Solution_IDx': solution_idx,
        'Starting_Node': network['starting_node'], 'Total_Reward': sum(RT),'Lookahead_Reward':max_tot_reward, 'Action_Trace': AT,
        'Node_Trace': NT, 'Reward_Trace': RT, 'Other_Reward_Trace':ORT,'Action_IDx':np.arange(8)}

In [142]:
def get_all_solutions(network):
    all_acts = get_actions(8)
    solution_list = []
    T,R=calculate_reward_transition_matrices_new(network,6)
    Q = calculate_q_matrix_avpruning(R, T, n_steps, gamma_g=0.0, gamma_s=0.0)
    max_tot_reward, aaa, bbb, ccc, ddd, eee = calculate_traces(Q, T, R)
    for i in range(2**8):
        solution_idx=i
        NT = np.zeros((n_steps + 1),dtype=np.int64)  # node trace: step
        AT = np.array(all_acts[i])  # action trace: step
        RT = np.zeros((n_steps),dtype=np.int64) # reward trace: step
        ORT = np.zeros((n_steps),dtype=np.int64) #other reward trace: step
        OAT = 1-AT
        starting_node = network['starting_node']
        NT[0] = starting_node
        for j in range(n_steps):
            if j<1:
                target_node = np.where(T[starting_node]>0.5)[0][AT[j]]
                reward = R[starting_node][AT[j]]
                other_reward = R[starting_node][OAT[j]]
            else:
                reward = R[target_node][AT[j]]
                other_reward = R[target_node][OAT[j]]
                target_node = np.where(T[target_node]>0.5)[0][AT[j]]
            NT[j+1] = target_node
            RT[j] = reward
            ORT[j] = other_reward
        solution_list.extend([make_row(network,solution_idx,RT,AT,NT,ORT,max_tot_reward[starting_node])])
    return solution_list

In [138]:
def df_all_sols(networks):
    all_sols = []
    for network in networks:
        sol_list = get_all_solutions(network)
        all_sols.extend(sol_list)
    df = pd.DataFrame(data=all_sols)
    return df

In [143]:
df = df_all_sols(networks)

In [145]:
df.head()

,Environment_ID,Solution_IDx,Starting_Node,Total_Reward,Lookahead_Reward,Action_Trace,Node_Trace,Reward_Trace,Other_Reward_Trace,Action_IDx
0,42_0,0,2,0,520,"[0, 0, 0, 0, 0, 0, 0, 0]","[2, 0, 1, 0, 1, 0, 1, 0, 1]","[-20, -100, 140, -100, 140, -100, 140, -100]","[-20, 20, 140, 20, 140, 20, 140, 20]","[0, 1, 2, 3, 4, 5, 6, 7]"
1,42_0,1,2,120,520,"[0, 0, 0, 0, 0, 0, 0, 1]","[2, 0, 1, 0, 1, 0, 1, 0, 3]","[-20, -100, 140, -100, 140, -100, 140, 20]","[-20, 20, 140, 20, 140, 20, 140, -100]","[0, 1, 2, 3, 4, 5, 6, 7]"
2,42_0,2,2,80,520,"[0, 0, 0, 0, 0, 0, 1, 0]","[2, 0, 1, 0, 1, 0, 1, 2, 0]","[-20, -100, 140, -100, 140, -100, 140, -20]","[-20, 20, 140, 20, 140, 20, 140, -20]","[0, 1, 2, 3, 4, 5, 6, 7]"
3,42_0,3,2,80,520,"[0, 0, 0, 0, 0, 0, 1, 1]","[2, 0, 1, 0, 1, 0, 1, 2, 1]","[-20, -100, 140, -100, 140, -100, 140, -20]","[-20, 20, 140, 20, 140, 20, 140, -20]","[0, 1, 2, 3, 4, 5, 6, 7]"
4,42_0,4,2,120,520,"[0, 0, 0, 0, 0, 1, 0, 0]","[2, 0, 1, 0, 1, 0, 3, 4, 3]","[-20, -100, 140, -100, 140, 20, 140, -100]","[-20, 20, 140, 20, 140, -100, -20, -100]","[0, 1, 2, 3, 4, 5, 6, 7]"


In [146]:
def expand_actions(df_walk, lst_col):
    '''expand lst_col (actions) column to rows. 1 row -> 8 rows '''
    df_walk_actions = pd.DataFrame({col: np.repeat(df_walk[col].values, df_walk[lst_col].str.len())
                                    for col in df_walk.columns.difference([lst_col])}).assign(**{lst_col: np.concatenate(df_walk[lst_col].values)})[df_walk.columns.tolist()]
    return df_walk_actions

In [152]:
def add_source(row):
    return row['Node_Trace'][row['Action_IDx']]

def add_target(row):
    return row['Node_Trace'][row['Action_IDx']+1]

def add_action_reward(row):
    return row['Reward_Trace'][row['Action_IDx']]

def add_other_reward(row):
    return row['Other_Reward_Trace'][row['Action_IDx']]

In [153]:
df_action=expand_actions(df,'Action_IDx')
df_action['Source_Node'] = df_action.apply(add_source,axis=1)
df_action['Target_Node'] = df_action.apply(add_target,axis=1)
df_action['Action_Reward'] = df_action.apply(add_action_reward,axis=1)
df_action['Other_Reward'] = df_action.apply(add_other_reward,axis=1)



In [154]:
df_action.head()

,Environment_ID,Solution_IDx,Starting_Node,Total_Reward,Lookahead_Reward,Action_Trace,Node_Trace,Reward_Trace,Other_Reward_Trace,Action_IDx,Source_Node,Target_Node,Action_Reward,Other_Reward
0,42_0,0,2,0,520,"[0, 0, 0, 0, 0, 0, 0, 0]","[2, 0, 1, 0, 1, 0, 1, 0, 1]","[-20, -100, 140, -100, 140, -100, 140, -100]","[-20, 20, 140, 20, 140, 20, 140, 20]",0,2,0,-20,-20
1,42_0,0,2,0,520,"[0, 0, 0, 0, 0, 0, 0, 0]","[2, 0, 1, 0, 1, 0, 1, 0, 1]","[-20, -100, 140, -100, 140, -100, 140, -100]","[-20, 20, 140, 20, 140, 20, 140, 20]",1,0,1,-100,20
2,42_0,0,2,0,520,"[0, 0, 0, 0, 0, 0, 0, 0]","[2, 0, 1, 0, 1, 0, 1, 0, 1]","[-20, -100, 140, -100, 140, -100, 140, -100]","[-20, 20, 140, 20, 140, 20, 140, 20]",2,1,0,140,140
3,42_0,0,2,0,520,"[0, 0, 0, 0, 0, 0, 0, 0]","[2, 0, 1, 0, 1, 0, 1, 0, 1]","[-20, -100, 140, -100, 140, -100, 140, -100]","[-20, 20, 140, 20, 140, 20, 140, 20]",3,0,1,-100,20
4,42_0,0,2,0,520,"[0, 0, 0, 0, 0, 0, 0, 0]","[2, 0, 1, 0, 1, 0, 1, 0, 1]","[-20, -100, 140, -100, 140, -100, 140, -100]","[-20, 20, 140, 20, 140, 20, 140, 20]",4,1,0,140,140
